In [1]:
from langchain_community.document_loaders import TextLoader

# Load text
loader = TextLoader("./data/genesi.txt")
content = loader.load()

In [2]:
# Define separators for text
separators = [
   ". ",
   "\n\n",
   "\n",
]

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create chunks
splitter = RecursiveCharacterTextSplitter(separators=separators,chunk_size=800, chunk_overlap=200)
docs = splitter.split_documents(content)

In [4]:
from langchain_ollama import OllamaEmbeddings

# Define and load an embedding model, nomic it's optmized for italian
embeddings_model = OllamaEmbeddings(model="mxbai-embed-large")

In [5]:
from langchain_qdrant import QdrantVectorStore
connection_string = "http://localhost:6333"

# Start embedding
qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embeddings_model,
    url=connection_string,
    collection_name="chap02",
    force_recreate=True,
    distance="Cosine"
)

In [6]:
retriever = qdrant.as_retriever(search_kwargs={"k": 4})

In [7]:
retriever.invoke("Testo biblico dove Dio proibisce di mangiare qualcosa")

[Document(metadata={'source': './data/genesi.txt', '_id': 'd9835125-4c7c-4371-a252-dd5c7c3779c3', '_collection_name': 'chap02'}, page_content='. 5\xa0Non ti chiamerai più Abramo, ma Abraamo, perché ti renderò padre di molte nazioni. 6\xa0Ti renderò molto, molto fecondo e ti farò diventare nazioni, e da te discenderanno dei re.\n7\xa0“Rispetterò il patto fra me e te e la tua discendenza dopo di te, di generazione in generazione; è un patto eterno, in virtù del quale io sono il tuo Dio e il Dio della tua discendenza dopo di te. 8\xa0E a te e ai tuoi discendenti dopo di te darò su base permanente il paese in cui hai vissuto come straniero, l’intero paese di Cànaan; e sarò il loro Dio”.\n9\xa0Poi Dio disse ad Abraamo: “Quanto a te, devi rispettare il mio patto, tu e la tua discendenza dopo di te, di generazione in generazione'),
 Document(metadata={'source': './data/genesi.txt', '_id': '30861d59-3396-4eab-8392-f7ed15a90204', '_collection_name': 'chap02'}, page_content='. Mise di proposito 

In [8]:
from numpy import dot
from numpy.linalg import norm

v1 = embeddings_model.embed_query("Dio proibisce di mangiare qualcosa")
v2 = embeddings_model.embed_query("ma il frutto dell'albero della conoscenza del bene e del male non devi mangiarlo")

similarity = dot(v1, v2) / (norm(v1) * norm(v2))
print(similarity)

0.6517329632403439
